###  Machine Learning e Data Science com Python de A à Z (Classificacão) - IA Expert Academy

In [ ]:
# Instalação

#!pip -q install plotly
#!pip -q install yellowbrick


In [ ]:
# Importando as bibliotecas
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

## Base de dados de crédito

- Fonte (adaptado): https://www.kaggle.com/laotse/credit-risk-dataset

### Exploração dos Dados

In [ ]:
df_credit = pd.read_csv('credit-data.csv')

df_credit.head() # Exibindo as primeiras linhas do dataset

#### Vamos analisar os atributos da base de dados de crédito e definir o tipo de variável de cada um deles.

- **Atributos:**
    - **clientid** - Id do cliente -> Variável categórica nominal
    - **Income** - Renda -> Variável numérica contínua
    - **Age** - Idade -> Variável numérica contínua
    - **Loan** - Dívida -> Variável numérica contínua
    - **c#default** - Pagou/ñ pagou emprestimo -> Variável numérica discreta -> **Variável target**
    
obs. default = 0 -> Pagou

obs. default = 1 -> Não pagou

#### Objetivo:

- Nosso objetivo é criar um modelo de Machine Learning para prever se um cliente irá pagar ou não o empréstimo de acordo com as variáveis presentes acima.

In [ ]:
df_credit.tail() # Exibindo as últimas linhas do dataset

In [ ]:
df_credit.describe() # Exibindo informações estatísticas do dataset

In [ ]:
df_credit[df_credit['income'] >= 69995.685578] # Exibindo os registros da pessoa com maior renda

### Visualização dos Dados

In [ ]:
# Exibindo a quantidade de registros para cada classe da variável default
np.unique(df_credit['default'], return_counts=True) 

#Resultado :

# 0: 1717 registros -> pagantes
# 1: 283 registros -> não pagantes
# 1° análise : A base de dados está desbalanceada, pois a quantidade de pagantes é muito maior que a quantidade de não pagantes.

In [ ]:
sns.countplot(x='default', data=df_credit);
plt.title('Quantidade de pagantes e não pagantes');

In [ ]:
plt.hist(df_credit['age'], bins=20, color='blue', edgecolor='black');
plt.title('Distribuição da idade');

In [ ]:
plt.hist(df_credit['income'], color='blue', edgecolor='black');
plt.title('Distribuição da renda');

In [ ]:
plt.hist(df_credit['loan'], color='blue', edgecolor='black');
plt.title('Distribuição do empréstimo');

In [ ]:
grafico = px.scatter_matrix(df_credit, dimensions=['age', 'income' ,'loan'], color='default')
grafico.show()
#scatter => gráfico de dispersão interessante para observar valores que estão fora do quadrante de normalidade

Após a análise pelo gráfico de dispersão, conseguimos encontrar alguns padrões de usuários não pagantes e também 3 anomalias na coluna 'Age' com idades negativas que iremos corrigir agora

### Tratamentos dos valores inconsistentes

In [ ]:
df_credit.loc[df_credit['age'] < 0]

#### Técnicas para tratar valores negativos

In [ ]:
# Apagar a coluna inteira (não recomendado quando se tem poucos dados inconsistentes)
df_credit2 = df_credit.drop('age' , axis=1)
df_credit2.head()

In [ ]:
# Apagar somente os registros com valores inconsistentes
indices = df_credit[df_credit['age'] < 0].index
df_credit3 = df_credit.drop(indices)
df_credit3

In [ ]:
# Validando se ainda existem valores de idade menores que 0
df_credit3.loc[df_credit3['age'] < 0]

In [ ]:
# Preencher os valores inconsistentes manualmente (Recomendado quando se tem poucos dados inconsistentes)

# Aqui iremos preencher o valor de idade negativo com a média das idades

df_credit['age'].mean()

# Nota-se que aqui a um problema, pois essa média calculada tem como unidade de medida os valores inconsistentes, 
#ou seja, a média está incorreta.

In [ ]:
# Achando a média real

df_credit['age'][df_credit['age'] > 0].mean() # Média é igual a 40.92

In [ ]:
# Preenchendo os valores inconsistentes com a média correta

df_credit.loc[df_credit['age'] < 0, 'age'] = 40.92

# Validando se ainda existem valores de idade menores que 0

df_credit.loc[df_credit['age'] < 0]

df_credit.head(27)

### Tratamento de Valores Faltantes

In [ ]:
df_credit.isnull().sum() # Verificando se existem valores nulos

df_credit.loc[pd.isnull(df_credit['age'])]

In [ ]:
# Preencher os registros nulos com a média

df_credit['age'].fillna(df_credit['age'].mean() , inplace=True)

In [ ]:
# Validando se os Valores Nulos foram registrados com a Média

df_credit.loc[(df_credit['clientid']== 29) | (df_credit['clientid']== 30) | (df_credit['clientid']== 31)]

### Divisão da base de dados em treino e teste (Previsores e Classe)

Normalmente chama-se a base de dados de previsores de X e a base de dados de classe de y.

In [ ]:
# Iremos pegar todos os registros das colunas "income" , "age" e "loan"
x_credit = df_credit.iloc[ : , 1:4].values
x_credit # Exibindo os valores, nota-se que foi convertido os valores de dataframe do pandas para array do numpy
         # Isto foi feito com o ".values" para que converta pois os modelos de ML só conseguem trabalhar com arrays do numpy

In [ ]:
# Pegando os valores da coluna "default" que é a nossa variável alvo

y_credit = df_credit.iloc[ : , 4].values
y_credit

### Escalonamento dos Valores

In [ ]:
x_credit

In [ ]:
x_credit[:,0].min() , x_credit[:,1].min() , x_credit[:,2].min()

In [ ]:
x_credit[: , 0].max() , x_credit[: , 1].max() , x_credit[: , 2].max()

Comparando os valores vimos que algumas variáveis são muito maiores que outras e vice-versa , isso pode fazer com que na hora que iremos aplicar algoritmos de Machine learning encima desses dados ele possa se enviesar e achar que variáveis com valores maiores são mais importantes do que variáveis com valores menores, para isso vamos aplicar a técnica de escalonamento de variáveis.

In [ ]:
from sklearn.preprocessing import StandardScaler # Importando a biblioteca para normalização dos dados

scaler_credit = StandardScaler()
x_credit = scaler_credit.fit_transform(x_credit)

In [ ]:
x_credit[:,0].min() , x_credit[:,1].min() , x_credit[:,2].min()

In [ ]:
x_credit[: , 0].max() , x_credit[: , 1].max() , x_credit[: , 2].max()

Nota-se que agora os dados estão mais escalonados e os algoritmso de Machine Learning irão se comportar melhor.

## Base de Dados do Censo

In [ ]:
df_census = pd.read_csv('census.csv')
df_census

### Exploração de Dados



### Análise dos atributos da base de dados do censo e definição do tipo de variável de cada um deles.



- **Atributos:**
    - **age** - Idade -> Variável numérica discreta
    - **workclass** - Classe trabalhadora -> Variável categórica nominal
    - **final-weight** - Pontuação -> Variável numérica contínua 
    - **education** - Educação -> variavel categórica ordinal
    - **education-num** - Número de anos de educação -> Variável numérica discreta
    - **marital-status** - Estado civil -> Variável categórica nominal
    - **occupation** - Ocupação -> Variável categórica nominal
    - **relationship** - Relacionamento -> Variável categórica nominal
    - **race** - Raça -> Variável categórica nominal
    - **sex** - Sexo -> Variável categórica nominal
    - **capital-gain** - Ganho de capital -> Variável numérica contínua
    - **capital-loos** - Perda de capital -> Variável numérica contínua
    - **hour-per-week** - Horas de trabalho por semana -> Variável numérica contínua
    - **native-country** - País de origem -> Variável categórica nominal
    - **income** - Renda -> Variável categórica ordinal -> **Variável target**

#### Objetivo:

- Nosso objetivo é criar um modelo de Machine Learning para prever se a renda de uma pessoa é maior ou menor que 50k por ano de acordo com as variáveis presentes acima.

#### Saídas possíveis para o Income:

- **income <= 50k**
- **income > 50k**

In [ ]:
## Visualizando estatísticas

df_census.describe()

In [ ]:
# Verificando Valores Nulos
df_census.isnull().sum()

In [ ]:
np.unique(df_census['income'], return_counts=True)

Temos que 24720 pessoas ganham menos que 50k e 7841 pessoas ganham mais que 50k.

### Visualização dos Dados

In [ ]:
sns.countplot(x=df_census['income']);

# Através do gráfico podemos ver que a base de dados está desbalanceada!

In [ ]:
plt.hist(x=df_census['age'])

In [ ]:
plt.hist(x=df_census['education-num'])

In [ ]:
plt.hist(x=df_census['hour-per-week'])

In [ ]:
grafico = px.treemap(df_census,path=['workclass' , 'age'])
grafico.show()

In [ ]:
grafico = px.treemap(df_census,path=['occupation' , 'relationship' , 'age'])
grafico.show()

In [ ]:
grafico = px.parallel_categories(df_census, dimensions=['occupation' , 'relationship'])
grafico.show()

In [ ]:
grafico = px.parallel_categories(df_census, dimensions=[ 'workclass','occupation' , 'relationship', 'income'])
grafico.show()

In [ ]:
grafico = px.parallel_categories(df_census, dimensions=[ 'education','income'])
grafico.show()

### Divisão entre previsores e classe

In [ ]:
df_census.columns

In [ ]:
x_census = df_census.iloc[: , 0:14].values
y_census = df_census.iloc[: , 14].values

In [ ]:
# Validando colunas
x_census[0]

In [ ]:
x_census

In [ ]:
y_census

### Tratamento de atributos categóricos

#### Entendendo o LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder_teste = LabelEncoder()

In [ ]:
x_census[: ,1]

In [ ]:
teste = label_encoder_teste.fit_transform(x_census[: ,1])

In [ ]:
teste

Pode-se perceber que ao usar labelencoder ele transforma os valores possíveis das variáveis categóricas em valores numéricos.

#### Usando LabelEncoder em todas as variáveis categóricas

In [ ]:
label_encoder_workclass = LabelEncoder()
label_encoder_education = LabelEncoder()
label_encoder_marital = LabelEncoder()
label_encoder_occupation = LabelEncoder()
label_encoder_relationship = LabelEncoder()
label_encoder_race = LabelEncoder()
label_encoder_sex = LabelEncoder()
label_encoder_country = LabelEncoder()

In [ ]:
x_census[0]

In [ ]:
x_census[: ,1] = label_encoder_workclass.fit_transform(x_census[: ,1])
x_census[: ,3] = label_encoder_education.fit_transform(x_census[: ,3])
x_census[: ,5] = label_encoder_marital.fit_transform(x_census[: ,5])
x_census[: ,6] = label_encoder_occupation.fit_transform(x_census[: ,6])
x_census[: ,7] = label_encoder_relationship.fit_transform(x_census[: ,7])
x_census[:, 8] = label_encoder_race.fit_transform(x_census[:, 8])
x_census[:, 9] = label_encoder_sex.fit_transform(x_census[:, 9])
x_census[:, 13] = label_encoder_country.fit_transform(x_census[:, 13])

In [ ]:
x_census[0]

In [ ]:
x_census

Ok, agora que transformamos todas as variáveis categóricas em numéricas, vamos aplicar o OneHotEncoder para que o algoritmo de Machine Learning não entenda que uma variável é maior que a outra por conta do valor numérico.

#### OneHotEncoder

o OneHotEncoder transforma as variáveis categóricas em variáveis binárias. criando uma coluna para cada valor possível da variável categórica. a coluna que tiver o valor 1 é a que representa o valor da variável categórica, o restante das colunas terão o valor 0.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
onehotencoder_census = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), 
                                                       [1 , 3 , 5 , 6 , 7 , 8 , 9 , 13 ])],
                                                       remainder='passthrough')



In [ ]:
x_census = onehotencoder_census.fit_transform(x_census).toarray()

In [ ]:
x_census

In [ ]:
x_census[0]

In [ ]:
x_census.shape


pode-se perceber que agora temos 108 colunas, isso acontece porque o OneHotEncoder criou uma coluna para cada valor possível das variáveis categóricas. Logo, todo objeto do nossso dataset agora é representado por 108 colunas.

#### Escalonamento dos valores

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler_census = StandardScaler()
x_census = scaler_census.fit_transform(x_census)

In [ ]:
x_census[0]